In [1]:
import pandas as pd
df = pd.read_csv('BTCUSDT.csv')

In [2]:
print(df['timestamp'].dtype)

object


In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')  # Use 'coerce' to handle invalid parsing

In [4]:
df['year'] = df['timestamp'].dt.year.astype(int)
df['month'] = df['timestamp'].dt.month.astype(int)
df['day'] = df['timestamp'].dt.day.astype(int)
df['hour'] = df['timestamp'].dt.hour.astype(int)
df['minute'] = df['timestamp'].dt.minute.astype(int)
df['day_name'] = df['timestamp'].dt.day_name()

In [5]:
df['max'] = df[['open', 'high', 'low', 'close']].max(axis=1)
df['min'] = df[['open', 'high', 'low', 'close']].min(axis=1)

In [6]:
df = df.iloc[1:,:]

In [8]:
df = df[df['year'].isin([2023,2024])]

In [ ]:
# Logic to generate 15 minute dataframe
dfs = []

for _, group in df.groupby(['year','month', 'day']):
    for i in range(0, len(group), 15):
        dfs.append(group.iloc[i:i+15])

df_15m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'timestamp': [df.iloc[0].timestamp],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_15m = pd.concat([df_15m, ohlc_data])

In [9]:
# Logic to generate 1 hour dataframe
dfs = []

for _, group in df.groupby(['year','month', 'day']):
    for i in range(0, len(group), 60):
        dfs.append(group.iloc[i:i+60])

df_15m = pd.DataFrame()

for df in dfs:
    ohlc_data = pd.DataFrame({
    'high' : [df['max'].max()],
    'low' : [df['min'].min()],
    'open' : [df.iloc[0].open],
    'close' : [df.iloc[-1].close],
    'timestamp': [df.iloc[0].timestamp],
    })

    ohlc_data.index = pd.to_datetime([f"{int(df.iloc[-1].year)}-{int(df.iloc[-1].month)}-{int(df.iloc[-1].day)}"])
    df_15m = pd.concat([df_15m, ohlc_data])

In [10]:
df_15m['year'] = df_15m['timestamp'].dt.year.astype(int)
df_15m['month'] = df_15m['timestamp'].dt.month.astype(int)
df_15m['day'] = df_15m['timestamp'].dt.day.astype(int)
df_15m['hour'] = df_15m['timestamp'].dt.hour.astype(int)
df_15m['minute'] = df_15m['timestamp'].dt.minute.astype(int)
df_15m['day_name'] = df_15m['timestamp'].dt.day_name()
df_15m['max'] = df_15m[['open', 'high', 'low', 'close']].max(axis=1)
df_15m['min'] = df_15m[['open', 'high', 'low', 'close']].min(axis=1)
df_15m['difference'] = df_15m['max'] - df_15m['min']
df_15m['candle_body'] = df_15m['open'] - df_15m['close']
df_15m['high_shadow'] = df_15m['high'] - df_15m[['open', 'close']].max(axis=1)
df_15m['low_shadow'] = df_15m['low'] - df_15m[['open', 'close']].min(axis=1)
df_15m['low-open'] = df_15m['low'] - df_15m['open']
df_15m['high-open'] = df_15m['high'] - df_15m['open']
df_15m['trend'] = df_15m['candle_body'].apply(lambda x: "UP" if x < 0 else "ZERO" if x == 0 else "DOWN")
df_15m['7_EMA'] = df_15m['close'].ewm(span=7, adjust=False).mean()
df_15m['15_EMA'] = df_15m['close'].ewm(span=15, adjust=False).mean()
df_15m['50_EMA'] = df_15m['close'].ewm(span=50, adjust=False).mean()
df_15m['200_EMA'] = df_15m['close'].ewm(span=200, adjust=False).mean()

In [11]:
df_15m

,high,low,open,close,timestamp,year,month,day,hour,minute,...,candle_body,high_shadow,low_shadow,low-open,high-open,trend,7_EMA,15_EMA,50_EMA,200_EMA
2023-01-01,16545.70,16508.39,16541.77,16529.67,2023-01-01 00:00:00,2023,1,1,0,0,...,12.10,3.93,-21.28,-33.38,3.93,DOWN,16529.670000,16529.670000,16529.670000,16529.670000
2023-01-01,16556.80,16525.78,16529.59,16551.47,2023-01-01 01:00:00,2023,1,1,1,0,...,-21.88,5.33,-3.81,-3.81,27.21,UP,16535.120000,16532.395000,16530.524902,16529.886915
2023-01-01,16559.77,16538.14,16551.47,16548.19,2023-01-01 02:00:00,2023,1,1,2,0,...,3.28,8.30,-10.05,-13.33,8.30,DOWN,16538.387500,16534.369375,16531.217651,16530.069036
2023-01-01,16548.19,16518.21,16548.19,16533.04,2023-01-01 03:00:00,2023,1,1,3,0,...,15.15,0.00,-14.83,-29.98,0.00,DOWN,16537.050625,16534.203203,16531.289116,16530.098598
2023-01-01,16535.97,16511.92,16533.04,16521.85,2023-01-01 04:00:00,2023,1,1,4,0,...,11.19,2.93,-9.93,-21.12,2.93,DOWN,16533.250469,16532.659053,16530.918954,16530.016522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-05,62100.00,61980.33,62039.52,62058.00,2024-10-05 23:00:00,2024,10,5,23,0,...,-18.48,42.00,-59.19,-59.19,60.48,UP,61970.324578,62000.154258,61854.504018,62471.002231
2024-10-06,62088.17,61888.46,62058.01,61920.00,2024-10-06 00:00:00,2024,10,6,0,0,...,138.01,30.16,-31.54,-169.55,30.16,DOWN,61957.743433,61990.134975,61857.072488,62465.519622
2024-10-06,62150.00,61919.99,61919.99,62081.01,2024-10-06 01:00:00,2024,10,6,1,0,...,-161.02,68.99,0.00,0.00,230.01,UP,61988.560075,62001.494353,61865.854352,62461.693655
2024-10-06,62115.99,61940.11,62081.01,61966.01,2024-10-06 02:00:00,2024,10,6,2,0,...,115.00,34.98,-25.90,-140.90,34.98,DOWN,61982.922556,61997.058809,61869.782024,62456.761480


In [37]:
df_15m_2023_and_2024 = df_15m[df_15m['year'].isin([2023,2024])]

In [13]:
df_15m.to_csv('BTC_USD_1_HOUR_DF.csv',index=True)